In [ ]:
exit(0)

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
!unzip tweets.zip

Archive:  tweets.zip
   creating: tweets/Chiellini/
  inflating: tweets/Chiellini/0.csv  
  inflating: tweets/Chiellini/1.csv  
  inflating: tweets/Chiellini/10.csv  
  inflating: tweets/Chiellini/11.csv  
  inflating: tweets/Chiellini/12.csv  
  inflating: tweets/Chiellini/13.csv  
  inflating: tweets/Chiellini/14.csv  
  inflating: tweets/Chiellini/15.csv  
  inflating: tweets/Chiellini/16.csv  
  inflating: tweets/Chiellini/17.csv  
  inflating: tweets/Chiellini/18.csv  
  inflating: tweets/Chiellini/19.csv  
  inflating: tweets/Chiellini/2.csv  
  inflating: tweets/Chiellini/20.csv  
  inflating: tweets/Chiellini/21.csv  
  inflating: tweets/Chiellini/22.csv  
  inflating: tweets/Chiellini/23.csv  
  inflating: tweets/Chiellini/24.csv  
  inflating: tweets/Chiellini/25.csv  
  inflating: tweets/Chiellini/26.csv  
  inflating: tweets/Chiellini/27.csv  
  inflating: tweets/Chiellini/28.csv  
  inflating: tweets/Chiellini/29.csv  
  inflating: tweets/Chiellini/3.csv  
  inflating: twe

In [ ]:
!pip install transformers -q

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 67.5 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 66.8 MB/s 
     |████████████████████████████████| 596 kB 77.8 MB/s 


In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [ ]:
!pip install --upgrade pandas==1.3.4

In [ ]:
# imports pytorch
import torch
import torch_xla
import torch_xla.core.xla_model as xm
from tqdm.notebook import trange, tqdm
import glob
import os
import pandas as pd

In [ ]:
dev = xm.xla_device()
dev

device(type='xla', index=1)

In [ ]:
def load_tweets(base_path, columns=None):
    if columns is None:
        columns = ['id', 'time', 'retweet_count', 'content', 'player']

    tweets_batch = []
    for file in glob.glob(base_path):
        current_data = pd.read_csv(file, sep=";")
        current_data['player'] = file.split(os.path.sep)[-2].split(".csv")[0]
        tweets_batch.append(current_data)

    data = pd.concat(tweets_batch)
    data = data[columns]
    data.drop_duplicates(subset=['id'], inplace=True, keep=False)
    return data

In [ ]:
tweets = load_tweets("tweets/*/*.csv")

In [ ]:
from scipy.special import softmax

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import pickle


def normalize_score(negative, neutral, positive):
    return ((positive + neutral / 2) * 2) - 1 if positive > negative else -((negative + neutral / 2) * 2) + 1


def preprocess(text):
    text = text[2:-2]
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


In [ ]:
task = 'sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(dev)

In [ ]:
print(len(tweets))

996999


In [ ]:
scores_map = {}
irony_map = {}

In [ ]:
batch_size = 4096
batches = np.array_split(tweets, len(tweets) // batch_size)

In [ ]:
for batch in tqdm(batches):
    ids = batch['id'].values
    texts = batch['content'].map(lambda x: preprocess(x)).values

    encoded_input =  tokenizer(list(texts),return_tensors='pt',padding=True,truncation=True, max_length=128)
    data_input_ids = encoded_input['input_ids'].to(dev)
    data_attention_mask = encoded_input['attention_mask'].to(dev)

    output = model.forward(input_ids=data_input_ids,attention_mask=data_attention_mask)

    scores = softmax(output.logits.cpu().detach().numpy(), axis=1)

    for idx, score in enumerate(scores):
        scores_map[ids[idx]] = normalize_score(*score)
        
with open('scores.pkl', 'wb') as file:
    pickle.dump(scores_map, file, pickle.HIGHEST_PROTOCOL)

  0%|          | 0/243 [00:00<?, ?it/s]

In [ ]:
print(len(scores_map))

996999


In [ ]:
task = 'irony'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(dev)

In [ ]:
for batch in tqdm(batches):
    ids = batch['id'].values
    texts = batch['content'].map(lambda x: preprocess(x)).values

    encoded_input =  tokenizer(list(texts),return_tensors='pt',padding=True,truncation=True, max_length=128)
    data_input_ids = encoded_input['input_ids'].to(dev)
    data_attention_mask = encoded_input['attention_mask'].to(dev)

    output = model.forward(input_ids=data_input_ids,attention_mask=data_attention_mask)

    scores = softmax(output.logits.cpu().detach().numpy(), axis=1)

    for idx, score in enumerate(scores):
      irony = score[1]
      irony_map[ids[idx]] = irony
        
with open('irony.pkl', 'wb') as file:
    pickle.dump(irony_map, file, pickle.HIGHEST_PROTOCOL)

  0%|          | 0/243 [00:00<?, ?it/s]